In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
events_data = pd.read_csv('event_data_train.csv')
submission_data = pd.read_csv('submissions_data_train.csv')

In [4]:

events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's')
events_data['day'] = events_data.date.dt.date

submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit = 's')
submission_data['day'] = submission_data.date.dt.date

users_data = events_data.groupby('user_id', as_index= False)\
        .agg({'timestamp': 'max'})\
        .rename({'timestamp':'last_timestamp'}, axis = 'columns')
now = 1526772811
drop_out_threasold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold
users_scores = submission_data.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()
users_data = users_data.merge(users_scores, on = 'user_id', how = 'outer')
users_data = users_data.fillna(0)
users_invent_data = events_data.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
users_data = users_data.merge(users_invent_data, how = 'outer')
users_days = events_data.groupby('user_id').day.nunique()
users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')
users_data['passed_corse'] = users_data.passed > 170

In [5]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False


In [11]:
user_min_time = events_data.groupby('user_id' , as_index=False).agg({'timestamp':'min'}).rename({'timestamp':'min_timestamp'} , axis=1)

In [12]:
users_data = users_data.merge(user_min_time , how='outer')

In [13]:
learning_time_threshold = 3 * 24 * 60 * 60

In [19]:
event_data_train = events_data.merge(users_data[['user_id', 'min_timestamp']], on='user_id', how='left') \
    .query(f"(timestamp - min_timestamp) < {learning_time_threshold}")

In [20]:
event_data_train.head()

,step_id,timestamp,action,user_id,date,day,min_timestamp
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,1434340848
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,1434340848


In [28]:
wrong_steps = submission_data[submission_data.submission_status == "wrong"].groupby(['user_id', 'step_id'], as_index=False)

In [30]:
wrong_steps.agg({'timestamp':'max'}).step_id.value_counts().keys()[0]

31978

In [35]:
submission_data = submission_data.merge(user_min_time, on='user_id', how='left')

In [39]:
# Время от первого степа до последнего
submission_data['users_time'] = submission_data['timestamp'] - submission_data['min_timestamp']
#Выбираем степы первых трёх дней
submissions_data_train = submission_data[submission_data.users_time <= learning_time_threshold]
submissions_data_train.groupby('user_id').day.nunique().max()

4

In [47]:
X = submissions_data_train.groupby('user_id').day.nunique().to_frame().reset_index() \
    .rename(columns={'day' : 'days'})

In [49]:
steps_tried = submissions_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index() \
              .rename(columns={'step_id' : 'steps_tried'})

In [50]:
X = X.merge(steps_tried, on='user_id', how='outer')

In [52]:
X = X.merge(submissions_data_train.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0) \
                        .reset_index())

In [54]:
X['correct_ration'] = X.correct / (X.correct + X.wrong)
X

,user_id,days,steps_tried,correct,wrong,correct_ration
0,2,1,2,2,0,1.000000
1,3,1,4,4,4,0.500000
2,8,1,11,9,21,0.300000
3,14,1,1,0,1,0.000000
4,16,3,23,23,27,0.460000
...,...,...,...,...,...,...
8908,26787,1,3,3,0,1.000000
8909,26790,1,1,1,0,1.000000
8910,26794,3,27,31,9,0.775000
8911,26797,1,2,2,0,1.000000


In [55]:
X = X.merge(event_data_train.pivot_table(index='user_id',\
                        columns = 'action',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0) \
                        .reset_index()[['user_id', 'viewed']], how='outer')

In [57]:
X = X.fillna(0)

In [77]:
users_data

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse,min_timestamp,min_+_3_days
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False,1472827464,1473086664
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False,1514383364,1514642564
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False,1434358476,1434617676
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False,1466156809,1466416009
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False,1521634660,1521893860
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19229,26790,1517495987,True,1.0,0.0,8,8,1,9,4,False,1451484016,1451743216
19230,26793,1510225034,True,0.0,0.0,1,0,1,1,1,False,1510225031,1510484231
19231,26794,1512035039,True,33.0,9.0,69,69,34,180,9,False,1495287049,1495546249
19232,26797,1509995096,True,2.0,0.0,10,10,2,13,2,False,1508425316,1508684516


In [59]:
X = X.merge(users_data[['user_id', 'passed_corse', 'is_gone_user']], how='outer')

In [61]:
X = X[~((X.is_gone_user == False) & (X.passed_corse == False))]

In [62]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ration,viewed,passed_corse,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [64]:
y = X.passed_corse
X = X.drop(['passed_corse', 'is_gone_user'], axis=1)

In [66]:
X = X.set_index(X.user_id)
X = X.drop(['user_id'], axis=1)

In [70]:
y = y.map(int)
y

0        0
1        0
2        0
3        0
4        1
        ..
19229    0
19230    1
19231    0
19232    0
19233    0
Name: passed_corse, Length: 17985, dtype: int64